In [1]:
import pandas as pd

# Faktencheck

## Vorstände - Einzeldaten mit Bonuszahlungen

Daten zusammengetragen aus den Jahresberichten / Offenlegungsberichten der Sparkassen

In [2]:
df = pd.read_csv('./data/jahresabschluss_vorstand.csv')
df_orig = df
df_bemessung = pd.read_csv('./data/bemessung.csv')
df = pd.merge(df, df_bemessung, on='sparkasse')

### Wie hoch ist der Anteil erfolgsabhängiger Gehälter am Gesamtgehalt? 

Die erfolgsabhängigen Bezüge für einen Vorstand werden immer erst im nächsten Jahr ausgewiesen, daher gelten die Werte von erfolgsabhaengige_bezuege von 2014 für den Zeitraum von 2013, in dem diejenige Person im Vorstand war.

In [3]:
# convert to date
df.since = pd.to_datetime(df.since)
df.until = pd.to_datetime(df.until)

# set year
df['year'] = df.until.map(lambda x: x.year)

# get "feste_bezuege" of year before
def get_salary(row):
    if row.year == 2014:
        lookup = df[(df.name == row['name']) & (df.year == 2013) & (df.sparkasse == row['sparkasse'])]
        if len(lookup) == 1:
            return lookup.iloc[0].feste_bezuege

df['feste_bezuege_2013'] = df.apply(get_salary, axis=1)

# calculate quota
df['bezuege_quote'] = df.erfolgsabhaengige_bezuege / df.feste_bezuege_2013

# calculate diff to erfolgs bezuege if quota would be .15
df['erfolgs_bezuege_diff'] = df.erfolgsabhaengige_bezuege - df.feste_bezuege_2013*0.15

# only relevant data
df = df[(df.bezuege_quote > 0) & (df.year == 2014)]

# shortcut for often used cols
columns = ['sparkasse', 'name', 'position', 'feste_bezuege', 'erfolgsabhaengige_bezuege', 'bezuege_quote', 'erfolgs_bezuege_diff']

**"In Nordrhein-Westfalen haben sieben Sparkassenvorstände ungewöhnlich hohe Bonuszahlungen erhalten. Die Prämien aus dem Jahr 2014 übetreffen den empfohlenen Grenzwert."**

**"Trotzdem hielten sich sieben Vorstände im Jahr 2014 nicht an diese Richtlinie"**

In [4]:
len(df[(df.bezuege_quote > 0.15) & (df.state == 'Nordrhein-Westfalen')].index)

7

**"Im Nachbarland Hessen sind die Richtlinien geheim. Dort liegen sogar 32 Vorstände über dem in NRW gültigen Grenzwert."**

In [5]:
len(df[(df.bezuege_quote > 0.15) & (df.state == 'Hessen')].index)

32

**"In Hessen lag sogar der Bonus von mehr als jedem zweiten Sparkassenchef über diesem Richtwert."**

In [6]:
df[df.state == 'Hessen'].bezuege_quote.quantile(0.5)

0.16336256195196341

**"Die Sparkasse KölnBonn hat ihrem kompletten Vorstand einen Bonus von gut 22 Prozent genehmigt. Der Vorsitzende Artur Grzesiek bekam neben einem Festgehalt von 590.600 Euro im Jahr 2013 noch weitere 137.500 Euro als Prämie ausgezahlt. Das sind fast 50.000 Euro mehr, als die beiden Sparkassenverbände in NRW als Richtwert festgeschrieben haben."**

In [7]:
df[df.sparkasse == 'Sparkasse KölnBonn'][columns]

,sparkasse,name,position,feste_bezuege,erfolgsabhaengige_bezuege,bezuege_quote,erfolgs_bezuege_diff
688,Sparkasse KölnBonn,Dr. Christoph Siemons,Mitglied,409500.0,91700.0,0.223932,30275.0
689,Sparkasse KölnBonn,Dr. Joachim Schmalzl,Mitglied,409500.0,91700.0,0.223932,30275.0
690,Sparkasse KölnBonn,Ulrich Voigt,Mitglied,409500.0,91700.0,0.223932,30275.0
692,Sparkasse KölnBonn,Artur Grzesiek,Vorsitzende(r),590600.0,137500.0,0.232814,48910.0


**"Direkt nebenan, in Leverkusen, erhielt der ehemalige Chef Manfred Herpolsheimer satte 26 Prozent seines Festgehaltes als Bonus. Macht 2014 allein 103.000 Euro zusätzlich – 42.000 Euro mehr als vorgesehen."**

In [8]:
df[df.sparkasse == 'Sparkasse Leverkusen'][columns]

,sparkasse,name,position,feste_bezuege,erfolgsabhaengige_bezuege,bezuege_quote,erfolgs_bezuege_diff
337,Sparkasse Leverkusen,Rainer Schwarz,Mitglied,324300.0,31600.0,0.131502,-4445.0
339,Sparkasse Leverkusen,Markus Grawe,Mitglied,331000.0,42100.0,0.131398,-5960.0
341,Sparkasse Leverkusen,Manfred Herpolsheimer,Vorsitzende(r),416600.0,103000.0,0.255266,42475.0


**"Die Sparkasse Märkisches Sauerland Hemer-Menden bezahlte ihrem Vorsitzenden einen Bonus von 23 Prozent. Dietmar Tacke, schon seit 1999 im Vorstand, bekam 62.000 als Prämie. Der Betrag liegt 22.000 Euro über dem vorgesehenen Grenzwert."**

In [9]:
df[df.sparkasse == 'Sparkasse Märkisches Sauerland Hemer-Menden'][columns]

,sparkasse,name,position,feste_bezuege,erfolgsabhaengige_bezuege,bezuege_quote,erfolgs_bezuege_diff
244,Sparkasse Märkisches Sauerland Hemer-Menden,Jörg Kötter,Mitglied,265000.0,25000.0,0.110619,-8900.0
247,Sparkasse Märkisches Sauerland Hemer-Menden,Dietmar Tacke,Vorsitzende(r),348000.0,62000.0,0.230483,21650.0


**"Die Stadtsparkasse Haltern am See zählt mit rund 100 Mitarbeitern zu den kleinsten Sparkassen in Deutschland. Dementsprechend ist das Gehalt der Vorstände vergleichsweise überschaubar. Im Vorstand sitzt Jutta Kuhn, die für ihre Arbeit im Jahr 2013 113.000 Euro bekam. Der Bonus von 23.000 Euro entspricht einem Fünftel Ihres Grundgehaltes. Das sind 6.050 Euro mehr als die beiden Sparkassen-Verbände in NRW empfehlen."**

In [10]:
df[df.sparkasse == 'Stadtsparkasse Haltern am See'][columns]

,sparkasse,name,position,feste_bezuege,erfolgsabhaengige_bezuege,bezuege_quote,erfolgs_bezuege_diff
69,Stadtsparkasse Haltern am See,Jutta Kuhn,Mitglied,112000.0,23000.0,0.203540,6050.0
71,Stadtsparkasse Haltern am See,Helmut Kanter,Vorsitzende(r),156000.0,11000.0,0.072368,-11800.0


In [11]:
df_bemessung.sort_values('bemessung_2014')[:13]

,sparkasse,bemessung_2014
416,Stadtsparkasse Bad Sachsa,347965698.0
14,Stadtsparkasse Felsberg,424412175.0
2,Stadtsparkasse Borken,434537800.0
1,Stadtsparkasse Grebenstein,506842930.0
4,Stadtsparkasse Schwalmstadt,537982370.0
0,Sparkasse Battenberg,556846950.0
415,Sparkasse Fröndenberg,624047144.0
5,Sparkasse Hohenwestedt,653971310.0
3,Sparkasse Laubach-Hungen,670256386.0
8,Sparkasse Geseke,736237869.0


**"Für die Sparkasse Märkisches Sauerland war 2013 ein schwieriges Jahr. Sie hielt knapp die schwarze null und machte nur 120.000 Euro Gewinn. Im Folgejahr waren fast 5 Prozent aller Unternehmenskredite notleidend."**

In [12]:
df_kerndaten = pd.read_csv('./data/jahresabschluss_kerndaten.csv')
df_kerndaten[df_kerndaten.sparkasse == 'Sparkasse Märkisches Sauerland Hemer-Menden'][[
    'sparkasse',
    'jahresueberschuss_2013',
    'npl_quote_unternehmen_2014'
]]

,sparkasse,jahresueberschuss_2013,npl_quote_unternehmen_2014
120,Sparkasse Märkisches Sauerland Hemer-Menden,120000.0,4.97


**"Auffällig ist in Hessen: [...]. Davon bewegen sich die Boni von 28 Vorständen bis an die 20 Prozent Grenze."**

In [13]:
len(df[(df.state == 'Hessen') & (df.bezuege_quote > 0.15) & (df.bezuege_quote <= 0.2)].index)

28

**"Eine Ausnahme ist der vierköpfige Vorstand der Frankfurter Sparkasse mit Bonuszahlungen zwischen 55 und 68 Prozent."**

In [14]:
df[df.sparkasse == 'Frankfurter Sparkasse'][columns]

,sparkasse,name,position,feste_bezuege,erfolgsabhaengige_bezuege,bezuege_quote,erfolgs_bezuege_diff
678,Frankfurter Sparkasse,Altfried M. Lütkenhaus,Mitglied,363000.0,200000.0,0.560224,146450.0
681,Frankfurter Sparkasse,Stephan Bruhn,Mitglied,368000.0,200000.0,0.561798,146600.0
682,Frankfurter Sparkasse,Arne Weick,Mitglied,369000.0,200000.0,0.550964,145550.0
683,Frankfurter Sparkasse,Herbert Hans Grüntker,Vorsitzende(r),452000.0,300000.0,0.678733,233700.0


**"In einer neuen Datenbank von correctiv.org und FAZ finden sich hunderte Gehälter von Vorständen aus 134 Sparkassen."**

In [15]:
len(df.index), len(df_orig[df_orig.gesamtverguetung > 0].sparkasse.unique())

(219, 134)

**"Mittlerweile veröffentlicht fast jede dritte Sparkasse ihre Vorstandgehälter. Im Jahr 2014 hatten 134 der damals noch 417 Sparkassen (jetzt 409) ihre Gehälter veröffentlicht."**

In [16]:
[len(df_orig[df_orig.gesamtverguetung > 0].sparkasse.unique()) / len(df_orig.sparkasse.unique()),
 len(df_kerndaten.sparkasse.unique())]

[0.32211538461538464, 417]

**"In NRW sind es 102 von 105 Sparkassen,..."**

In [17]:
[len(df_orig[(df_orig.state == 'Nordrhein-Westfalen') & (df_orig.gesamtverguetung > 0)].sparkasse.unique()), 
 len(df_orig[df_orig.state == 'Nordrhein-Westfalen'].sparkasse.unique())]

[102, 105]

**"...in Hessen 30 von 34."**

In [18]:
[len(df_orig[(df_orig.state == 'Hessen') & (df_orig.gesamtverguetung > 0)].sparkasse.unique()),
 len(df_orig[df_orig.state == 'Hessen'].sparkasse.unique())]

[30, 34]

**"Dazu veröffentlichen die Sparkassen in München und Berlin ihre Gehälter."**

In [19]:
'Stadtsparkasse München' in df_orig[df_orig.gesamtverguetung > 0].sparkasse.values

True

In [20]:
'Berliner Sparkasse' in df_orig[df_orig.gesamtverguetung > 0].sparkasse.values

True

**"Von den übrigen, schweigenden Sparkassen veröffentlicht rund die Hälfte zumindest das Gesamtgehalt aller Vorstände."**

In [21]:
sparkassen_schweigend = set(df_kerndaten.sparkasse.values) - set(df_orig[df_orig.gesamtverguetung > 0].sparkasse.values)
sparkassen_gesamtgehalt = sparkassen_schweigend & set(df_kerndaten[df_kerndaten.gesamtbezuege_2014 > 0].sparkasse.values)
len(sparkassen_gesamtgehalt) / len(sparkassen_schweigend)

0.5406360424028268